In [4]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from clust import *
import umap
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import umap
import hdbscan
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
from hyperopt import fmin, tpe, space_eval, Trials, STATUS_OK, partial
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials


# Analysis

## Load the corpus

In [9]:
corpus = pd.read_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_disto.csv")
corpus["thought"]
corpus_neg = list(corpus["thought"])
corpus


,Unnamed: 0,thought,original_label,cleaned_thought
0,0,Someone I trusted stole something valuable of ...,emotional reasoning,someone trusted stole something valuable mine ...
1,1,She doesn't respect me.,overgeneralizing,doesnt respect
2,2,My friend is ignoring his recently-deceased wife.,disqualifying the positive,friend ignoring recentlydeceased wife
3,3,he took me for granted!,"labeling,catastrophizing",took granted
4,4,We will not be able to complete the work that ...,catastrophizing,able complete work need done
...,...,...,...,...
917,917,Not having made rent and having to borrow mone...,Labeling,made rent borrow money already debt mother fai...
918,918,I’m a loser I hate myself and I’m ready to die,Labeling,im loser hate im ready die
919,919,People around me do not need my presence I'm w...,Emotional reasoning,people around need presence im worthless
920,920,I realize that I still don’t have anyone who I...,Overgeneralizing,realize still dont anyone could spend time wit...


## Sentence Embedding
https://www.sbert.net/index.html

In [10]:
"""
Sentences are mapped to sentence embeddings with three different pre-trained model
https://huggingface.co/sentence-transformers/all-mpnet-base-v2
https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2
https://huggingface-co.translate.goog/sentence-transformers/all-distilroberta-v1?_x_tr_sl=en&_x_tr_tl=fr&_x_tr_hl=fr&_x_tr_pto=sc

"""
i=0

# specify the model
model = 'all-mpnet-base-v2'
model_2 = 'all-MiniLM-L12-v2'
model_3 = 'All-Distilroberta-v1'
models = [model,model_2,model_3]

# Load the model
embedder = SentenceTransformer(model)
embedder_2 = SentenceTransformer(model_2)
embedder_3 = SentenceTransformer(model_3)

# encode
corpus_embeddings = embedder.encode(corpus_neg)
corpus_embeddings_2 = embedder_2.encode(corpus_neg)
corpus_embeddings_3 = embedder_3.encode(corpus_neg)

# merge 3 embeding
embeddings = [corpus_embeddings, corpus_embeddings_2, corpus_embeddings_3]

for embedding in embeddings:
    print(f"Model: {models[i]}")
    print(f"Shape: {embedding.shape}")
    i=i+1


Model: all-mpnet-base-v2
Shape: (922, 768)
Model: all-MiniLM-L12-v2
Shape: (922, 384)
Model: All-Distilroberta-v1
Shape: (922, 768)


## Export embedding

In [11]:
index = 0
for embedding in embeddings:
    umap_data = umap.UMAP(n_components=2,random_state=42).fit_transform(embedding)
    x, y = umap_data.T
    corpus[f"{models[index]} x"] = x
    corpus[f"{models[index]} y"] = y
    index = index +1
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_embedding.csv")



/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


## Clustering


### K-mean clustering


In [12]:
# Parameters
n_clusters = 13 # number of cluster
dimension_reduction = True # reduce the dimension before clustering
dim = 3 # number of dimension to reduce embedding to.

In [13]:
index = 0
cluster_dict_k = {}

for embedder in embeddings:
    kmeans = 0
    print(f"run model {models[index]}")   
    corpus_embeddings_reduce = embedder

    # Dimension reduction
    if dimension_reduction:
        print(f"dimension before reduction {corpus_embeddings_reduce.shape}")
        # Reduce the embedding with UMAP
        model = UMAP(n_components=dim,random_state=42)
        corpus_embeddings_reduce = model.fit_transform(corpus_embeddings_reduce)
        print(f"dimension after reduction {corpus_embeddings_reduce.shape}")

    # Cluster with KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state = 42)
    kmeans.fit(corpus_embeddings_reduce)
    corpus[f"{models[index]} k-mean"] = kmeans.labels_
    print(kmeans.labels_)
    
    # Reduce the dimension with UMAP for visualization
    model2 = UMAP(n_components=2,random_state=42)
    embeddings_2d = model2.fit_transform(embedder)
    x, y = embeddings_2d.T  
    corpus[f"{models[index]} x"] = x
    corpus[f"{models[index]} y"] = y




    index = index + 1


run model all-mpnet-base-v2
dimension before reduction (922, 768)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


dimension after reduction (922, 3)
[ 9  7  0  0 12  5 10  5  0  2  1  9  0  2  4  4  8 12 11  2  6  5  3  7
  4  1  4  4  2 11 11 11  8  0 12  9  4 10 12  1 10 11  7  7  0  1 10  7
  7  7  0  4  4  2  9  6  0 12  1  2  6  6 10 12  4  4 12  9  5  5 12 12
 12  2  9  7 10  0 10 10 10  0 10  7  6 10 12  7  6  3  5 12  0  9 10  4
  6  4 12  5  2  8 12 10  2  1 12  6 10  3  5 10 10 12  5  1 10  6 11  4
  7 10  6 11  9  1 10  4 12 11  4 12  2  5 10  5  4 12  3  0  7  5  5  5
  5  3 12  5 10  6 10  0  0  0  6  9  5  0  6  6  3  8  9 12  9  4  2  0
 12  3 12  5  5 12  2  1 11  3  3  4  9 10  6 11  5  4 10 12 12  4  0  0
  1  0  0  0 11 12  7  7  7  9 12  8  0  0  0  4  4  1  7  7  7 11  7  2
  7  8  8 12  6  3  0 12  0  2  5  7 10  4  6 10  7  2  4  7  2  9  9  0
  0  9  9  7  7 11  7  5 11 12  5 12 12 10 12 10  4  4  8  0 11  0  6  5
 12  5  6 10  7 11  6  2 11  8 12  4  9  8 11  2 12  4  8  5 10  7  3  3
  3  3  3  3  3  3  4  4  7  9 12 12  1  1  1  6  1 11 11  4  3 12  3  3
  8  3 10 12 12 

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


run model all-MiniLM-L12-v2
dimension before reduction (922, 384)


...

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


run model All-Distilroberta-v1
dimension before reduction (922, 768)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


dimension after reduction (922, 3)
[ 1  4  4  6  8  7  3  7  6  2  0  4  1  0  2 12 12  0 11 12 11  7  3  4
 12  3  0  2  2  5 11 11  2  6  0  9  0 10  0  3  7 11  4  4  6  3  8  4
  4  4  4  0  0  2  9  5  6  0  3 12  5  1  3  8 11  0  7  9  7  8  0  0
  0  2  9  4 10  6  8  1  8  4  1  4  6  8  8  4  5  8  7  3  6  9  8 12
  5 12  0  7  2  2  0  1  2  3  0  7  8  5  8  8 10  8  7  3  1  6 11 11
  6  1  5  5  6  7 10 11  0 11 11  2  0  7  8  7  5  0 12  1  4  6  7  7
  8 10  0  7  7  8  1  6  6  6 12  9  7  6  5  5 10 12  7  8  9 12  2  6
  0  8  2  7  7  0  2  3 11 12 10 11  9  7  5  5  7  9  3  0  0 12  6  6
  0  6  6  6 11  0  4  4  4 12  0 12 12  6  1 12 12  5  4  4  4 11  4 12
  4 12 12  0  5  0  6  2  6 12  7  4  7  0  1  8  4 12 11  4  2  9  9  6
  6  9  6  4  4 11  4  8 11  8  8  7  0  1  0  1 12  9  2  6 12  7 11  7
  0  8 12  3  4 11  8 12 11  2  0  7  6  2 11  2  0 12  2  7  1  4  8  8
  8  8  8  8  8  8 12 12  6  9  0  0  3  3  0  8  3 11 11 12  5  0  8  3
  2  7 10  0  2 

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [14]:
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_kmean.csv")


### Aglomerative Clustering (Retirer)

In [54]:
# Parameters
dimension_reduction = True # reduce the dimension before clustering
dim = 3 # number of dimension to reduce embedding to.

In [55]:
index = 0
cluster_dict_k = {}

for embedder in embeddings:
    kmeans = 0
    print(f"run model {models[index]}")   
    corpus_embeddings_reduce = embedder

    # Dimension reduction
    if dimension_reduction:
        print(f"dimension before reduction {corpus_embeddings_reduce.shape}")
        # Reduce the embedding with UMAP
        model = UMAP(n_components=dim,random_state=42)
        corpus_embeddings_reduce = model.fit_transform(corpus_embeddings_reduce)
        print(f"dimension after reduction {corpus_embeddings_reduce.shape}")

    corpus_embeddings_reduce = corpus_embeddings_reduce /  np.linalg.norm(corpus_embeddings_reduce, axis=1, keepdims=True)

    # Cluster with KMeans
    clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5)
    clustering_model.fit(corpus_embeddings_reduce)
    corpus[f"{models[index]} aglo"] = clustering_model.labels_
    print(clustering_model.labels_)
    
    # Reduce the dimension with UMAP for visualization
    model2 = UMAP(n_components=2,random_state=42)
    embeddings_2d = model2.fit_transform(corpus_embeddings_reduce)
    x, y = embeddings_2d.T  
    corpus[f"{models[index]} x"] = x
    corpus[f"{models[index]} y"] = y




    index = index + 1


run model all-mpnet-base-v2
dimension before reduction (922, 768)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


dimension after reduction (922, 3)
[1 0 0 0 1 1 1 1 0 2 2 1 1 2 2 2 2 2 0 2 0 2 2 0 2 2 2 2 2 0 0 0 2 0 2 1 2
 1 2 1 1 0 0 0 0 2 1 0 0 0 0 2 2 2 1 0 0 2 1 2 0 1 1 1 0 2 2 1 1 1 2 2 2 2
 1 0 1 0 1 1 1 0 1 0 0 1 2 0 0 2 1 2 0 1 1 2 0 0 2 1 2 2 2 1 2 2 2 2 1 2 1
 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 2 0 0 2 2 1 1 1 0 2 1 0 0 2 1 1 1 1 2 1
 1 1 1 0 0 0 1 1 1 0 0 2 1 2 1 2 1 0 2 0 2 1 2 1 1 2 2 2 0 1 1 0 0 1 0 0 1
 2 1 1 2 2 0 0 2 0 0 0 0 2 0 0 0 1 2 2 0 0 0 2 2 2 0 0 0 0 0 2 0 2 2 2 0 1
 0 2 0 2 1 0 1 2 1 1 0 2 0 0 2 1 1 0 0 1 1 0 0 0 0 2 0 2 2 2 2 1 2 1 2 2 2
 0 0 2 0 1 2 1 2 1 0 0 1 2 0 2 2 2 1 2 0 2 2 2 2 1 1 0 1 1 1 1 1 1 1 1 2 2
 0 1 2 2 2 2 2 0 1 0 0 0 1 2 2 2 2 2 1 2 2 2 2 1 2 2 0 2 1 1 1 1 1 1 1 1 2
 1 1 0 1 1 2 2 2 2 1 1 1 0 0 0 2 0 1 1 1 1 1 1 1 2 0 1 1 1 1 1 1 1 1 0 1 1
 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 2 1 1 2 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1
 1 1 2 2 1 0 1 2 1 1 2 1 1 1 1 2 2 1 2 0 2 0 0 1 1 0 2 2 2 2 2 2 2 1 0 1 0
 0 2 2 1 1 0 2 0 1 2 2 2 2 2 0 2 2 2 0 0 0 1 1 0 1 0 0 1 0 2 0 0 

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


run model all-MiniLM-L12-v2
dimension before reduction (922, 384)


...

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


run model All-Distilroberta-v1
dimension before reduction (922, 768)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


dimension after reduction (922, 3)
[3 1 1 1 0 0 0 0 1 2 4 1 3 4 2 2 2 0 2 2 3 0 0 1 2 0 0 2 4 1 3 2 4 1 0 2 0
 3 4 0 0 2 1 1 1 0 0 1 1 1 1 0 0 2 1 3 1 0 0 2 3 3 0 0 2 0 0 2 0 0 0 4 4 2
 2 1 3 1 0 3 0 2 3 1 1 0 0 1 3 0 0 0 1 2 0 3 3 3 4 0 2 2 4 3 2 0 4 0 0 0 0
 0 3 0 0 0 3 1 1 2 1 3 3 1 1 0 3 2 4 3 3 2 2 0 0 0 3 4 2 3 1 1 0 0 0 3 4 0
 0 0 3 1 1 1 2 2 0 1 3 3 3 2 0 0 2 2 2 1 4 0 4 0 0 0 2 0 2 2 3 2 1 0 3 3 0
 2 0 0 4 2 1 1 4 1 1 1 2 4 1 1 1 2 4 2 2 1 3 2 2 3 1 1 1 3 1 2 1 2 2 4 3 0
 1 4 1 2 0 1 0 0 3 0 1 2 2 1 2 1 2 1 1 2 1 1 1 2 1 0 2 0 0 0 4 3 4 3 3 2 2
 1 2 0 3 0 4 0 2 0 1 3 0 2 3 2 0 0 1 4 2 2 4 2 2 0 3 1 0 0 0 0 0 0 0 0 3 2
 1 2 0 0 0 0 4 0 0 3 3 2 3 4 0 0 4 0 3 4 2 2 4 0 4 2 2 4 3 3 4 2 0 0 0 0 0
 3 3 1 3 3 0 0 0 0 0 3 3 3 3 3 2 3 3 3 0 0 0 0 0 2 3 0 0 3 3 3 3 0 3 3 3 3
 3 1 1 0 2 2 1 1 1 1 3 3 0 0 3 1 3 2 3 3 3 3 3 1 1 1 1 1 2 2 2 1 1 3 3 3 3
 3 3 0 0 3 3 0 0 0 2 4 3 0 0 0 2 4 1 2 1 2 3 2 0 3 1 2 0 2 0 4 2 2 3 1 0 1
 1 0 4 0 1 1 2 2 4 0 4 0 0 0 1 2 0 0 1 1 1 0 3 2 2 1 3 2 1 0 1 3 

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [56]:
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_aglom.csv")


### DBSCAN

In [18]:
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_hdbscan.csv")


### Bayesian Estimation

In [19]:
best_params_dict = {}
best_clusters_dict = {}
trials_dict = {}

hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(2,20)),
    "n_components": hp.choice('n_components', range(2,20)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,8)),
    "random_state": 42
}

label_lower = 5
label_upper = 60
max_evals = 200

In [21]:
index=0
for embedding in embeddings:
    embedding_name = f"embedding_{index}"
    best_params_use, best_clusters_use, trials_use = bayesian_search(embedding, 
                                                                    space=hspace, 
                                                                    label_lower=label_lower, 
                                                                    label_upper=label_upper, 
                                                      max_evals=max_evals)
    best_params_dict[embedding_name] = best_params_use
    best_clusters_dict[embedding_name] = best_clusters_use
    trials_dict[embedding_name] = trials_use
    cluster_dict[f"label_use{index}"] =    best_clusters_dict[embedding_name]
    corpus[f"{models[index]} hdbscan_bayesian_optimisation"] = best_clusters_use.labels_

    umap_data = umap.UMAP(n_neighbors=15, 
                          random_state=42).fit_transform(embedding)
    x, y = umap_data.T
    corpus[f"{models[index]} x"] = x
    corpus[f"{models[index]} y"] = y
    index=index+1


  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")



  0%|          | 1/200 [00:01<05:11,  1.57s/trial, best loss: 0.403470715835141]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")



  1%|          | 2/200 [00:03<04:59,  1.51s/trial, best loss: 0.3495661605206074]

...

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")



100%|█████████▉| 199/200 [04:28<00:01,  1.32s/trial, best loss: 0.12689804772234273]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")



100%|██████████| 200/200 [04:29<00:00,  1.35s/trial, best loss: 0.12689804772234273]
best:
{'min_cluster_size': 5, 'n_components': 16, 'n_neighbors': 3, 'random_state': 42}
label count: 50


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [23]:
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_hdbscan_bayesian_optimisation.csv")
